In [ ]:
#COSAS QUE NO USO
def re_obtain_punteos_efectos(txt):
    reg = r"(?<=  - ).*?(?=:)|(?<= - ).*?(?=:)"
    vals = re.findall(reg, txt)
    return(vals)

def re_info_punteos_efectos(txt,punto):
    reg = r"(?<=" + punto + r": )(.*?)(?= - )"
    valu = re.findall(reg, txt)
    if len(valu) > 0:
        return(valu[0])
    return('N/A')

def re_obtain_punteos_precauciones(txt):
    text_c = re.sub(r' {2,}',r' ', txt)
    reg = r"(?<=: )\w*?(?=:)|(?<=\. )\w*?(?=: )|(?<=\. )Pacientes con enfermedad.*?(?=:)|(?<=\. )Conducir.*?(?=:)|(?<=\. )Consumo de .*?(?=: )"
    valus = re.findall(reg, txt)
    if len(valus)>0:
        return(valus)
    else:
        return('N/A')

def re_info_punteos_precauciones(txt,puntos):
    valus = {}
    for i in range(len(puntos)-1):
        punto = puntos[i]
        if i < len(puntos)-1:
            punto_sig = puntos[i+1]
            reg = r"(?<="+punto+r": ).*?(?="+punto_sig+r")"
        else:
            reg = r"(?<=" + punto + r": ).*"
        valus[punto] = re.findall(reg,txt)
        if(len(valus[punto])):
            valus[punto] = valus[punto][0]
    return(valus)

In [2]:
import pandas as pd
import os
import requests
import re
import fitz

ruta_nombres = "C:\\Users\\liada\\OneDrive\\Imágenes\\Medicamento\\"
archivo = "MedicamentosChile.csv"
ruta_archivo = "http://www.ispch.gob.cl/sites/default/files/5_agencia_reguladora/folletos/doc/enero2009/"
ext = '.pdf'
all_meds = pd.read_csv(ruta_nombres + archivo, delimiter = ";", encoding = 'UTF-8')

#Para descargar el pdf de acuerdo al url obtenido
def download_pdf(url : str):
    response = requests.get(url, stream=True, verify=False)
    pdf_file_name = os.path.basename(url)
    if response.status_code == 200:
        filepath = os.path.join(os.getcwd(), pdf_file_name)
        with open(filepath, 'wb') as pdf_object:
            pdf_object.write(response.content)
        return(True)
    else:
        return(False)
#Para eliminar los \n y se pega el texto de todas las páginas
def get_formated_text(med):
    doc = fitz.open(med+ext)
    text = ""
    for page in doc:
        text += page.get_text()
    text_c = re.sub(r'\n', '', text)
    text_c = str(text_c)
    return(text_c)
#Para el formato 1, sin comic sans
def get_dict_regex_f1(med):
    regex_dict = {}
    med = med.replace('+ ', '/')
    largo_med = len(med.strip()) + med.count(' ')
    regex_dict["uso"]= r"(?<=PARA QUE SE USA .{"+str(largo_med)+r"}\?)(.*?)(?=\¿)"
    regex_dict["restricciones"] = r"(?<=CUÁNDO NO USAR .{" +str(largo_med) + r"}\?)(.*?)(?=AD)"
    regex_dict["advertencias"] = r"(?<=ADVERTENCIAS)(.*?)(?=PR)"
    regex_dict["precauciones"] = r"(?<=DE USO)(.*?)(?=¿)"
    regex_dict["interacciones"] = r"(?<=QUÉ INTERACCIONES MEDICAMENTOSAS PRESENTA .{"+ str(largo_med) + r"}\?)(.*?)(?=¿)"
    regex_dict["administracion"] = r"(?<=CÓMO ADMINISTRAR Y EN QUE CANTIDAD\?)(.*?)(?=EFECTOS NO DESEADOS:)"
    regex_dict["efectos-no-deseados"] = r"(?<=EFECTOS NO DESEADOS:)(.*?)(?=¿)"
    regex_dict["enfermedades-previas"] = r"(?<=¿QUÉ SUCEDE SI SUFRE DE OTRA ENFERMEDAD ANTES DE COMENZAR EL TRATAMIENTO\?)(.*?)(?=SOBREDOSIS)"
    regex_dict["sobredosis"] = r"(?<=SOBREDOSIS:)(.*?)(?=¿)"
    regex_dict["almacenamiento"] = r"(?<=CÓMO ALMACENAR\?)(.*?)(?=[A-Z]{2})"
    return(regex_dict)
#para el formato 2, con comic sans
def get_dict_regex_f2(med):
    regex_dict = {}
    regex_dict['uso'] = r"(?<=Para qué se usa\?).*?(?=\d\.-)"
    regex_dict['administracion'] = r"(?<=Administración).*?(?=\d\.-)"
    regex_dict['precauciones'] = r"(?<=Precauciones).*?(?=\d\.-)"
    regex_dict['restricciones'] = r"(?<=Usted no debe tomar este medicamento en los siguientes casos: ).*?(?=\d\.-)"
    regex_dict['interacciones'] = r"(?<=Interacciones).*?(?=\d\.-)"
    regex_dict['efectos-no-deseados'] = r"(?<=Efectos adversos).*?(?=\d\.-)"
    regex_dict['sobredosis'] = r"(?<=Sobredosis).*?(?=\d\.-)"
    regex_dict['almacenamiento'] = r"(?<=almacenamiento).*?(?= No)"
    regex_dict['advertencias'] = r"(?<=Advertencias)(.*?)(?=\d\.-)"
    regex_dict['enfermedades-previas'] = r"(?<=Enfermedades).*?(?=\d\.-)|(?<=Enfermedades).*?(?=- )"
    return(regex_dict)
    
#Para obtener la info de cada regex    
def get_val_from_re(text,reg):
    val = re.findall(reg,text)
    if(len(val)>0):
        return(val[0])
    else:
        return('N/A')

In [7]:
#Prueba
med = all_meds['Medicamentos'][4]
url_pdf = ruta_archivo+med+ext
download_pdf(url_pdf)

C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


'Exito'

In [3]:
text = get_formated_text(med)
med_dict = get_dict_regex(med)

NameError: name 'med' is not defined

In [11]:
efectos_no_deseados = str(get_val_from_re(text, med_dict['sobredosis']))+" - "
puntos = re_obtain_punteos_efectos(efectos_no_deseados)
dict_info_puntos = {}
for p in puntos:
    dict_info_puntos[p] = re_info_punteos_efectos(efectos_no_deseados, p)
print(dict_info_puntos)

{}


In [12]:
precauciones = str(get_val_from_re(text, med_dict["precauciones"]))
puntos = re_obtain_punteos_precauciones(precauciones)
valus = re_info_punteos_precauciones(precauciones,puntos)
print(precauciones)

 Embarazo y lactancia: El uso de alendronato sódico durante el embarazo y la lactancia no ha sido bien estudiado, por lo que su uso está contraindicado. Uso pediátrico. La seguridad y eficacia en pacientes pediátricos no han sido establecidas, por lo que su uso está contraindicado. Uso geriátrico. En los estudios clínicos no hubo ninguna diferencia relacionada con la edad en la eficacia o la seguridad de Alendronato. Efectos indeseables a nivel del esófago: Esofagitis, úlceras y erosiones esofagianas han sido informadas en los pacientes tratados con Alendronato. En algunos casos estas reacciones han sido severas y han necesitado hospitalización. El médico debe por consiguiente estar atento en cuanto a la existencia de todo síntoma o signo que indique una posible reacción esofagiana. Pacientes con insuficiencia renal: Ninguna modificación es necesaria en pacientes de edad ni en pacientes que tengan una insuficiencia renal ligera a moderada. Calcio: Los pacientes tratados deben ser suple

# Ciclo de algunos de los medicamentos para chequear que esté todo bien

In [4]:
first_5 = all_meds['Medicamentos'].copy()
dict_data_f_5 = {}
for m in first_5:
    m = str(m)
    if '+' in m:
        me = m.replace('+', '/')
    else:
        me = m
    dict_data_f_5[m] = {}
    url_pdf = ruta_archivo+m+ext
    if(download_pdf(url_pdf)):
        text = get_formated_text(m)
        regex_dict_1 = get_dict_regex_f1(me)
        regex_dict_2 = get_dict_regex_f2(me)
        test = get_val_from_re(text,regex_dict_1['uso'])
        if test != 'N/A': 
            reg_dict = regex_dict_1
        else:
            reg_dict = regex_dict_2
        for r in reg_dict.keys():
            dict_data_f_5[m][r] = get_val_from_re(text,reg_dict[r])
        os.remove(m+ext)
    else:
        dict_data_f_5[m]['N/A'] = 'N/A'

C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\conn

C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\conn

C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\conn

C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\conn

C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ispch.gob.cl'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\liada\anaconda3\Lib\site-packages\urllib3\conn

KeyboardInterrupt: 

In [13]:
for k in dict_data_f_5.keys():
    print(k)

Abacavir
Abatacept
Acenocumarol
ACIDO ACETILSALICILICO+ DIPIRIDAMOL
Acido alendronico
ÁCIDO UNDECILÉNICO
ACITRETINA
5 metil aminolevulinato _clorhidrato_ crema topica 16%
A.A.S
Adapaleno topica 1%
ADENOSINA
Adefovir dipivoxilo comprimidos 10 mg
PARACETAMOL
ACICLOVIR_SISTEMICO
ACICLOVIR_TOPICO
ACIDO_ACETILSALICILICO_CAFEINA_CINAMEDRINA
ACIDO_ACETILSALICILICO_I
ACIDO_ACETILSALICILICO_II
ACIDO_ASCORBICO
ACIDO_ASCORBICO_CAFEINA_CLORFENAMINA_NOSCAPINA_PARACETAMOL
ACIDO_ASCORBICO_NOSCAPINA_PARACETAMOL_PSEUDOEFEDRINA
ACIDO_CITRICO_BICARBONATO_DE_SODIO
ACIDO_DEHIDROCOLICO
ACIDO_FOLICO
ACIDO_LACTICO_ACIDO_SALICILICO
ACIDO_MEFENAMICO
ACIDO_NALIDIXICO
ACIDO_SALICILICO
ACIDO_SALICILICO_AMILO_ACETATO
ACIDO_SALICILICO_AZUFRE
ACIDO_VALPROICO
Acido Valproico _sal magnesica_ 250mg-500 mg oral
Alfa-1-antitripsina
ALANTOINA_CETILPIRIDINIO
ALBENDAZOL
ALCANFOR_CALAMINA_DIFENHIDRAMINA
ACETAZOLAMIDA
ALCANFOR_CLOROXILENO_MENTOL
ALBÚMINA HUMANA
ALOPURINOL
ALPRAZOLAM
AMFOTERICINA-B_
Anfebutamona
AMBROXOL
AMILOR

# Pasar a json

In [23]:
# -*- coding: utf-8 -*-
import json

with open('C:\\Users\\liada\\OneDrive\\Imágenes\\Medicamento\\test.json', 'w', encoding='utf8') as f:
    data = json.dumps(dict_data_f_5,ensure_ascii=False).encode('utf-8')
    f.write(str(data, 'utf-8'))
    